This notebooks finetunes VGG16 by adding a new layes trains it to classify between cats and dogs

In [1]:
import numpy as np

import tensorflow as tf

from tensorflow.contrib.keras import layers
from tensorflow.contrib.keras import models
from tensorflow.contrib.keras import optimizers
from tensorflow.contrib.keras import applications
from tensorflow.contrib.keras.python.keras.preprocessing import image
from tensorflow.contrib.keras.python.keras.applications import imagenet_utils

In [11]:
vgg16 = applications.VGG16(weights="imagenet", include_top=False, input_shape=(224,224,3))

In [12]:
vgg16.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [13]:
def get_batches(dirpath, gen=image.ImageDataGenerator(), shuffle=True, batch_size=64, class_mode='categorical'):
    return gen.flow_from_directory(dirpath, target_size=(224,224), class_mode=class_mode, shuffle=shuffle, batch_size=batch_size)

In [14]:
batch_size = 64

In [15]:
train_batches = get_batches('./data/train', batch_size=batch_size)

Found 22778 images belonging to 2 classes.


In [16]:
val_batches = get_batches('./data/valid', batch_size=batch_size)

Found 2222 images belonging to 2 classes.


In [23]:
finetune_in = vgg16.output

In [24]:
flat = layers.Flatten()(finetune_in)

In [25]:
predictions = layers.Dense(train_batches.num_class, activation='softmax')(flat)

In [26]:
model = models.Model(inputs=vgg16.input, outputs=predictions)

In [27]:
for layer in model.layers[:-1]:
    layer.trainable = False

In [28]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [29]:
lr = 0.01

In [30]:
model.compile(optimizer=optimizers.Adam(lr=lr), loss='categorical_crossentropy', metrics=['accuracy'])

In [35]:
epochs = 1

In [36]:
model.fit_generator(train_batches, validation_data=val_batches, epochs=epochs,
                    steps_per_epoch=train_batches.samples // train_batches.batch_size, 
                    validation_steps=val_batches.samples // val_batches.batch_size)

Epoch 1/1
  1/355 [..............................] - ETA: 14161s - loss: 9.3426 - acc: 0.3125

KeyboardInterrupt: 